# Lab:  Model Order Selection for Neural Data

Machine learning is a key tool for neuroscientists to understand how sensory and motor signals are encoded in the brain.  In addition to improving our scientific understanding of neural phenomena, understanding neural encoding is critical for brain machine interfaces.  In this lab, you will use model selection for performing some simple analysis on real neural signals.  

Before doing this lab, you should review the ideas in the [polynomial model selection demo](./polyfit.ipynb).  In addition to the concepts in that demo, you will learn to:
* Represent neural time-series data in arrays
* Load data from a pickle file
* Describe and fit memoryless linear models
* Describe and fit linear time-series models with delays
* Fit linear models with multiple target outputs
* Select the optimal delay via cross-validation

## Loading the data

The data in this lab comes from neural recordings described in:

<a href="http://jn.physiology.org/content/106/2/764.short">
Stevenson, Ian H., et al. "Statistical assessment of the stability of neural movement representations." Journal of neurophysiology 106.2 (2011): 764-774</a>

Neurons are the basic information processing units in the brain.  Neurons communicate with one another via *spikes* or *action potentials* which are brief events where voltage in the neuron rapidly rises then falls.  These spikes trigger the electro-chemical signals between one neuron and another.  In this experiment, the spikes were recorded from 196 neurons in the primary motor cortex (M1) of a monkey using an electrode array implanted onto the surface of a monkey's brain.  During the recording, the monkey performed several reaching tasks and the position and velocity of the hand was recorded as well.  

The goal of the experiment is to try to *read the monkey's brain*:  That is, predict the hand motion from the neural signals from the motor cortex.

We first load the key packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

The full data is available on the CRCNS website  http://crcns.org/data-sets/movements/dream.  This website has a large number of great datasets and can be used for projects as well.  However, the raw data files can be quite large.  To make the lab easier, the [Kording lab](http://kordinglab.com/) at UPenn has put together an excellent [repository](https://github.com/KordingLab/Neural_Decoding) where they have created simple pre-processed versions of the data.  You can download the file `example_data_s1.pickle` from the [Dropbox link](https://www.dropbox.com/sh/n4924ipcfjqc0t6/AADOv9JYMUBK1tlg9P71gSSra/example_data_s1.pickle?dl=0).  Alternatively, you can directly run the following code.  This may take a little while to download since the file is 26 MB.

In [ ]:
fn_src = 'https://www.dropbox.com/sh/n4924ipcfjqc0t6/AADOv9JYMUBK1tlg9P71gSSra/example_data_s1.pickle?dl=1'
fn_dst = 'example_data_s1.pickle'

import os
from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s is already downloaded' % fn_dst)
else:
    urllib.request.urlretrieve(fn_src, fn_dst)

The file is a *pickle* data structure, which is a package to serialize python objects into data files.  Once you have downloaded the file, you can run the following command to retrieve the data from the pickle file.

In [ ]:
with open('example_data_s1.pickle', 'rb') as fp:
    X,y = pickle.load(fp)

The matrix `X` is matrix of spike counts where `X[i,j]` is the number of spikes from neuron `j` in time bin `i`.
The matrix `y` has two columns:
* `y[i,0] = ` velocity of the monkey's hand in the x-direction
* `y[i,1] = ` velocity of the monkey's hand in the y-direction
Our goal will be to predict `y` from `X`.  

Each time bin represent `tsamp=0.05` seconds of time.  Using `X.shape` and `y.shape` compute and print:
* `nt = ` the total number of time bins
* `nneuron = ` the total number of neurons
* `nout = ` the total number of output variables to track = number of columns in `y`
* `ttotal = ` total time of the experiment is seconds.

In [ ]:
tsamp = 0.05   # sampling time in seconds

# TODO
nt = X.shape[0]            # number of time bins
nneurons = X.shape[1]      # number of neurons
nout = y.shape[1]          # number of output variables
ttotal = nt * tsamp        # total time in seconds

print("nt =", nt)
print("nneurons =", nneurons)
print("nout =", nout)
print("ttotal =", ttotal, "seconds")


## Fitting a Memoryless Linear Model

Let's first try a simple linear regression model to fit the data.

First, use the `train_test_split` function to split the data into training and test.  Let `Xtr,ytr` be the training data set and `Xts,yts` be the test data set.  Use `test_size=0.33` so `1/3` of the data is used for test.

In [ ]:
from sklearn.model_selection import train_test_split

# TODO
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=0.33, random_state=0)

print("Training set:", Xtr.shape, ytr.shape)
print("Test set:", Xts.shape, yts.shape)


Now, fit a linear model using `Xtr,ytr`.  Make a prediction  `yhat` using `Xts`.  Compare `yhat` to `yts` to measure `rsq`, the  `R^2`.  You can use the `r2_score` method. Print the `rsq` value.  You should get `rsq` of around `0.45`.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Fit linear regression model
model = LinearRegression()
model.fit(Xtr, ytr)

# TODO
yhat = model.predict(Xts)          # predictions on test set
rsq = r2_score(yts, yhat)          # R^2 score

print("R^2 =", rsq)


It is useful to plot the predicted vs. true values.  Since we have two outputs, create two `subplots` using the `plt.subplot()` command.  In plot `i=0,1`, plot `yhat[:,i]` vs. `yts[:,i]` with a scatter plot.   Label the axes of the plots.  You may also use the command:

    plt.figure(figsize=(10,5))
    
to make the figures a little larger.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))

for i in range(y.shape[1]):
    plt.subplot(1, 2, i+1)
    plt.scatter(yts[:, i], yhat[:, i], alpha=0.5)
    plt.xlabel("True y{}".format(i))
    plt.ylabel("Predicted y{}".format(i))
    plt.plot([yts[:, i].min(), yts[:, i].max()],
             [yts[:, i].min(), yts[:, i].max()],
             'r--')

plt.tight_layout()
plt.show()


## Fitting Models with Delay

One way we can improve the model accuracy is to used delayed version of the features.  Specifically, the model we used above mapped the features

    yhat[i,k] = \sum_{j=0}^{p-1} X[i,j]*w[j,k] + b[k]
    
where `p` is the number of features and `w[j,k]` is a matrix of coefficients.  In this model,  `yhat[i,:]` at time `i` was only dependent on the inputs  `X[i,:]` at time `i`.  In signal processing, this is called a *memoryless* model.  However, in many physical systems, such as those that arise in neuroscience, there is a delay between the inputs `X[i,:]` and the outputs `y[i]`.  For such cases, we can use a model of the form,

    yhat[i+d,k] = \sum_{j=0}^{p-1} \sum_{m=0}^d X[i+m,j]*W[j,m,k] + b[k]
    
where `W` is a 3-dim array of coefficients where:

    W[j,m,k] is the influence of the input X[i+m,j] onto output y[i+d,k]


In signal processing, this model is called an *FIR* filter and `W[j,:,k]` is the *impulse response* from the `j`-th input to the `k`-th output.  The point is that the output at time `i+d` depends on the inputs at times `i,i+1,...,i+d`.  Hence, it depends on the last `d+1` time steps, not just the most recent time.

To translate this into a linear regression problem, complete the following function that creates a new feature and target matrix where:

    Xdly[i,:] has the rows X[i,:], X[i++1,:], ..., X[i+dly,:]
    ydly[i,:] = y[i+dly,:]
    
Thus, `Xdly[i,:]` contains all the delayed fetaures for the target `yhat`.  Note that if `X` is `n x p` then `Xdly` will be `n-dly x (dly+1)*p`.  


In [ ]:
import numpy as np

def create_dly_data(X, y, dly):
    """
    Create delayed data
    X: (n, p) input features
    y: (n, q) output
    dly: number of delays
    """
    nt, p = X.shape

    # 构造延迟特征
    Xdly = np.zeros((nt - dly, p * (dly + 1)))
    for i in range(dly, nt):
        Xdly[i - dly, :] = X[i - dly:i + 1, :].flatten(order='F')
        # order='F' 保证是按时间步拼接（X[i-dly], ..., X[i]）

    # 构造目标：从 dly 开始
    ydly = y[dly:,:]

    return Xdly, ydly



Now fit an linear delayed model with `dly=6` additional delay lags.  That is,
* Create delayed data `Xdly,ydly=create_dly_data(X,y,dly=6)`
* Split the data into training and test as before
* Fit the model on the training data
* Measure the `R^2` score on the test data

If you did this correctly, you should get a new `R^2` score around 0.69.  This is significantly better than the memoryless models.

In [ ]:
# Create delayed data with dly=6
Xdly, ydly = create_dly_data(X, y, dly=6)

# Split into training and test sets
Xtr, Xts, ytr, yts = train_test_split(Xdly, ydly, test_size=0.33, random_state=0)

# Fit linear regression model
model = LinearRegression()
model.fit(Xtr, ytr)

# Predict and compute R^2 score
yhat = model.predict(Xts)
rsq = r2_score(yts, yhat)

print("R^2 with delayed model (dly=6):", rsq)


Plot the predicted vs. true values as before. You should visually see a better fit.

In [ ]:
plt.figure(figsize=(10,5))

for i in range(y.shape[1]):   # 两个输出变量
    plt.subplot(1, 2, i+1)
    plt.scatter(yts[:, i], yhat[:, i], alpha=0.5)
    plt.xlabel("True y{}".format(i))
    plt.ylabel("Predicted y{}".format(i))
    plt.plot([yts[:, i].min(), yts[:, i].max()],
             [yts[:, i].min(), yts[:, i].max()],
             'r--')   # 参考对角线

plt.tight_layout()
plt.show()


*Note*:  Fitting an FIR model with the above method is very inefficient when the number of delays, `dly` is large.  In the above method, the number of columns of `X` grows from `p` to `(dly+1)*p` and the computations become expensive with `dly` is large.  We will describe a much faster way to fit such models using gradient descent when we talk about convolutional neural networks.

## Selecting the Optimal Delay via Model Order Selection

In the previous example, we fixed `dly=6`.  We can now select the optimal delay using model order selection.  Since we have a large number of data samples, it turns out that the optimal model order uses a very high delay.  Using the above fitting method, the computations take too long.  So, to simplify the lab, we will first just pretent that we have a very limited data set.

Compute `Xred` and `yred` by taking the first `nred=6000` samples of the data `X` and `y`.  This is about 10% of the overall data.

In [ ]:
nred = 6000

# TODO
Xred = X[:nred, :]
yred = y[:nred, :]

print("Reduced data shapes:", Xred.shape, yred.shape)


Now complete the following code to implement K-fold cross validation with `nfold=5` and values of delays `dtest = [0,1,...,dmax]`.  

The code also includes a progress bar using the `tqdm` package.  This is very useful when you have a long computation.

Note:  Some students appeared to use the `mse` metric (i.e. RSS per sample) instead of `R^2`.  That is fine.  For the solution, I have computed both.

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
import tqdm.notebook

nfold = 5     # number of folds
dmax = 15     # maximum number of delays

kf = model_selection.KFold(n_splits=nfold, shuffle=True, random_state=0)

# Model orders to be tested
dtest = np.arange(0, dmax+1)
nd = len(dtest)

# Initialize Rsq matrix: rows=folds, cols=model orders
Rsq = np.zeros((nfold, nd))

# Create progress bar
pbar = tqdm.notebook.tqdm(total=nd*nfold, initial=0,
                          unit='fits', unit_divisor=nd,
                          desc='Model order test')

for id, d in enumerate(dtest):
    # TODO: Create delayed data
    Xdly, ydly = create_dly_data(Xred, yred, d)

    # Loop over folds
    for ifold, Ind in enumerate(kf.split(Xdly)):
        Itr, Its = Ind   # training and test indices

        # TODO: Split data
        Xtr, Xts = Xdly[Itr,:], Xdly[Its,:]
        ytr, yts = ydly[Itr,:], ydly[Its,:]

        # TODO: Fit linear regression
        model = LinearRegression()
        model.fit(Xtr, ytr)

        # TODO: Predict and compute R^2
        yhat = model.predict(Xts)
        Rsq[ifold, id] = r2_score(yts, yhat)

        pbar.update(1)

pbar.close()


Compute the mean and standard error of the `R^2` values as a function of the model order `d`.  Use a `plt.errorbar` plot.  Label your axes.

In [ ]:
# Compute mean and standard error
Rsq_mean = np.mean(Rsq, axis=0)
Rsq_se = np.std(Rsq, axis=0) / np.sqrt(Rsq.shape[0])

# Plot with error bars
plt.figure(figsize=(8,5))
plt.errorbar(dtest, Rsq_mean, yerr=Rsq_se, fmt='-o', capsize=5)
plt.xlabel("Model order (dly)")
plt.ylabel("R^2 score")
plt.title("Model order selection")
plt.grid(True)
plt.show()



Find the optimal order `d` with the normal rule (i.e. highest test `R^2`)

In [ ]:
# Find optimal order d with normal rule (highest mean R^2)
best_d = dtest[np.argmax(Rsq_mean)]
best_r2 = np.max(Rsq_mean)

print("Optimal delay order (normal rule):", best_d)
print("Corresponding mean R^2:", best_r2)


Now find the optimal model order via the one SE rule (i.e. highest test `R^2` within on SE)

In [ ]:
# One standard error rule
best_mean = np.max(Rsq_mean)
best_idx = np.argmax(Rsq_mean)
threshold = best_mean - Rsq_se[best_idx]

# 选择满足 Rsq_mean >= threshold 的最小 d
candidate_idx = np.where(Rsq_mean >= threshold)[0]
best_d_se = dtest[np.min(candidate_idx)]

print("Optimal delay order (1-SE rule):", best_d_se)
print("Corresponding mean R^2:", Rsq_mean[np.min(candidate_idx)])
